<a href="https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/U01/MyFirstPyTorchNN-PimaIndian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="Bilder/ost_logo.png" width="240" height="120" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>


## Ein erstes Neuronales Netz mit `PyTorch`: Pima Indians

## (a) Datensatz laden

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import matplotlib.pyplot as plt

# Use GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the dataset
df = pd.read_csv('https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/U01/diabetes.csv')



## (b) Kurze EDA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool_)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))


# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, vmin=0.0, center=0, annot=True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});
plt.show()


In [ ]:
df.describe()

In [ ]:
X = df.iloc[:,0:8].values
y = df.iloc[:,8].values


## (c) Vorbereitung eines Datensatzes in PyTorch: Dataset-Klasse

In PyTorch sind Datensätze und DataLoader essenziell, um Daten effizient für das Training und die Evaluation von Modellen bereitzustellen. Hier wird beschrieben, wie man einen eigenen Datensatz vorbereitet, ihn in ein `Dataset`-Objekt einbindet und mit einem `DataLoader` verwendet.
Zuerst müssen die Daten in geeignete PyTorch-Tensoren umgewandelt werden.
- **`X`**: Enthält die Eingabe-Features (z. B. Spalten 0–7 eines DataFrames).
- **`y`**: Enthält die Zielwerte (z. B. Spalte 8 eines DataFrames).
- **`dtype=torch.float32`**: Konvertiert die Daten in den Gleitkomma-Datentyp, der häufig in PyTorch verwendet wird.
- **`.to(device)`**: Verschiebt die Tensoren optional auf die GPU oder CPU, je nach Bedarf.

In [ ]:

# Split into features (X) and target (y)
X = torch.tensor(df.iloc[:, 0:8].values, dtype=torch.float32).to(device)
y = torch.tensor(df.iloc[:, 8].values, dtype=torch.float32).unsqueeze(1).to(device)

#### **Erstellung einer benutzerdefinierten Dataset-Klasse**

PyTorch bietet die Möglichkeit, eigene Datensätze zu erstellen, indem man von der Klasse `torch.utils.data.Dataset` erbt. Dies ermöglicht die flexible Handhabung von Daten.

- **`__init__`**: Initialisiert die Klasse mit den Eingabedaten (`X`) und den Zielwerten (`y`).
- **`__len__`**: Gibt die Anzahl der Datenpunkte im Datensatz zurück.
- **`__getitem__`**: Ermöglicht den Zugriff auf die Datenpunkte über einen Index. Dies wird vom `DataLoader` verwendet, um Daten in Batches zu laden.


In [ ]:
# Custom Dataset class
class DiabetesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

## (d) Aufteilen in einen Trainings und Validierungsdatensatz & DataLoader

Teilen Sie den geladenen Datensatz in einen Trainings- und einen Testdatensatz auf (Aufteilung nach Zeilen) und verwenden Sie einen die Trainingsdaten (80%), um das Modell zu trainieren, und den Validierungsdatensatz (20%), um die Leistung des Modells auf neuen Daten zu schätzen. Erzeugen Sie einen `DataLoader` für die Trainings- und für die Validierungsdaten.


In [ ]:
# Create dataset and split into train and validation sets
dataset = DiabetesDataset(X, y)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])




Ein `DataLoader` wird verwendet, um Daten in Mini-Batches zu laden und optional zufällig zu mischen.
- **`batch_size`**: Die Anzahl der Datenpunkte pro Batch.
- **`shuffle=True`**: Mischt die Daten zu Beginn jeder Epoche.

Der DataLoader kann in der Trainingsschleife verwendet werden, um Mini-Batches zu laden:

```python
for batch in dataloader:
    inputs, targets = batch
    # Weiterverarbeitung (z. B. Modellvorhersage und Verlustberechnung)
    outputs = model(inputs)
    loss = loss_fn(outputs, targets)
    # Backpropagation und Optimierung
```

Dieses Vorgehen ermöglicht eine effiziente und skalierbare Verarbeitung von Datensätzen in PyTorch.

In [ ]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
batch_size = 8
print_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
inputs, labels = next(iter(print_loader))  # Get the first batch

labels

## (e) Modell erstellen

In [ ]:

# Define the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

model = Net().to(device)
# Assuming 'model' is your defined neural network
model = model.to(torch.float32)

In [ ]:
# Define the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8, 128)
        self.dropout1 = nn.Dropout(p=0.5)  # Dropout layer after fc1
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(p=0.5)  # Dropout layer after fc2
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)  # Apply dropout after activation
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)  # Apply dropout after activation
        x = torch.sigmoid(self.fc3(x))
        return x

model = Net().to(device)
# Assuming 'model' is your defined neural network
model = model.to(torch.float32)


## (f) Modell darstellen

In [ ]:
print(model)

In [ ]:
# Import required libraries
import torch
from torchsummary import summary
from torchviz import make_dot

# Summarize the model
summary(model, input_size=(1, 8))  # Adjust input_size as per your model's requirement

# Visualize the model architecture
x = torch.randn(1, 1, 8)  # Adjust input size as per your model's requirement
y = model(x)
make_dot(y, params=dict(model.named_parameters())).render("model_architecture", format="png")
from torchsummary import summary
from torchviz import make_dot

## (g) Modell kompilieren und trainieren

In [ ]:

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

# Training loop
epochs = 300
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)

        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        predicted = (output > 0.5).float()
        train_total += target.size(0)
        train_correct += (predicted == target).sum().item()

    train_losses.append(train_loss / len(train_loader))
    train_accuracies.append(100 * train_correct / train_total)

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(val_loader):
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()
            predicted = (output > 0.5).float()
            val_total += target.size(0)
            val_correct += (predicted == target).sum().item()

    val_losses.append(val_loss / len(val_loader))
    val_accuracies.append(100 * val_correct / val_total)

    print(f'Epoch [{epoch+1}/{epochs}], '
          f'Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accuracies[-1]:.2f}%, '
          f'Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracies[-1]:.2f}%')




In [ ]:
# Evaluierung des Modells (Beispiel)
correct = 0
total = 0
with torch.no_grad():
    for data, target in train_loader:
        output = model(data)
        predicted = (output > 0.5).float()
        total += target.size(0)
        correct += (predicted == target).sum().item()

print('Accuracy: {}'.format(100 * correct / total))

## (h) Modell abspeichern

Sie können Ihr Modell speichern, indem Sie die Funktion `save()` für das Modell aufrufen und den Dateinamen angeben.
Das folgende Beispiel demonstriert dies, indem zunächst ein Modell angepasst, ausgewertet und in der Datei "model.h5" gespeichert wird.

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'diabetes_model.pth')

## (i) Modell laden

In [ ]:
import torch
import torch.nn as nn


# Load the saved state dictionary
state_dict = torch.load('diabetes_model.pth')

# Create an instance of your model
model = Net()

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Now you can use the loaded model for inference or further training

## (j) Lernkurven grafisch darstellen

Die Funktion `fit()` gibt ein `History`-Objekt zurück, das den Verlust und die Genauigkeit am Ende jeder Epoche zusammenfasst. Erstellen Sie Liniendiagramme dieser Daten, sogenannte Lernkurven.


In [ ]:
# Plot learning curves
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.ylim(0.4, 0.8)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## (k) Trainieren auf einem neuen Datensatzes

Probieren Sie, einen anderen Tabellendatensatz zu verwenden, vielleicht aus dem UCI Machine Learning Repository.

## (l)  Das Modell anpassen

Ändern Sie die Konfiguration des Modells oder den Trainingsprozess und sehen Sie, ob Sie die Leistung des Modells verbessern können, z. B. eine Genauigkeit von mehr als 76 % erreichen.
